In [1]:
import random
import numpy as np
import torch
from math import floor
import os
import matplotlib.pyplot as plt
from glob import glob

In [2]:
def read_pdb(filename):
	
	with open(filename, 'r') as file:
		strline_L = file.readlines()
		# print(strline_L)

	X_list = list()
	Y_list = list()
	Z_list = list()
	atomtype_list = list()
	for strline in strline_L:
		# removes all whitespace at the start and end, including spaces, tabs, newlines and carriage returns
		stripped_line = strline.strip()

		line_length = len(stripped_line)
		# print("Line length:{}".format(line_length))
		if line_length < 78:
			print("ERROR: line length is different. Expected>=78, current={}".format(line_length))
		
		X_list.append(float(stripped_line[30:38].strip()))
		Y_list.append(float(stripped_line[38:46].strip()))
		Z_list.append(float(stripped_line[46:54].strip()))

		atomtype = stripped_line[76:78].strip()
		if atomtype == 'C':
			atomtype_list.append('h') # 'h' means hydrophobic
		else:
			atomtype_list.append('p') # 'p' means polar

	return X_list, Y_list, Z_list, atomtype_list

In [3]:
DATA_DIR='E:/Dataset/course/BS6207/training_data/'

In [4]:
def create_data_rec(pro, lig, label,
                    pro_list, lig_list, label_list, width, unit):
    file_name = "{}_{}_cg.pdb".format(("0000" + str(lig))[-4:], "lig")
    X_list, Y_list, Z_list, atomtype_list = read_pdb(DATA_DIR + file_name)
    x0 = floor(np.mean(X_list))
    y0 = floor(np.mean(Y_list))
    z0 = floor(np.mean(Z_list))
    all_atoms = list(zip(X_list, Y_list, Z_list, atomtype_list))
    use_atoms = [x for x in all_atoms if point_within_vox((x[0], x[1], x[2]),
                                                          x0 - width, x0 + width, y0 - width,
                                                          y0 + width, z0 - width, z0 + width)]
    
    lig_list.append(convert_xyz_to_vox(use_atoms, x0 - width, x0 + width, y0 - width,
                                       y0 + width, z0 - width, z0 + width, unit))
    
    
    
    file_name = "{}_{}_cg.pdb".format(("0000" + str(pro))[-4:], "pro")
    X_list, Y_list, Z_list, atomtype_list = read_pdb(DATA_DIR + file_name)
    all_atoms = list(zip(X_list, Y_list, Z_list, atomtype_list))
    use_atoms = [x for x in all_atoms if point_within_vox((x[0], x[1], x[2]),
                                                          x0 - width, x0 + width, y0 - width,
                                                          y0 + width, z0 - width, z0 + width)]
    pro_list.append(convert_xyz_to_vox(use_atoms, x0 - width, x0 + width, y0 - width,
                                       y0 + width, z0 - width, z0 + width, unit))
    label_list.append(label)

In [5]:
def point_within_vox(pt, x_lower=-50, x_upper=50, y_lower=-50,
                     y_upper=50, z_lower=-50, z_upper=50):
    (atom_x, atom_y, atom_z) = pt
    check_x = (atom_x >= x_lower) and (atom_x < x_upper)
    check_y = (atom_y >= y_lower) and (atom_y < y_upper)
    check_z = (atom_z >= z_lower) and (atom_z < z_upper)
    return check_x and check_y and check_z


def convert_xyz_to_vox(atoms_list,
                       x_lower=-50, x_upper=50, y_lower=-50,
                       y_upper=50, z_lower=-50, z_upper=50, unit=1):
    length = int((x_upper-x_lower)/unit)
    vox = np.zeros((length, length, length, 2))

    for atom in atoms_list:
        (x, y, z, t) = atom
        index_x = floor((x-x_lower)/unit)
        index_y = floor((y-y_lower)/unit)
        index_z = floor((z-z_lower)/unit)
        if t =='h':
            vox[index_x,index_y,index_z, 0] += 1
        elif t =='p':
            vox[index_x, index_y, index_z, 1] += 1

    return vox

In [6]:

# PLEASE CHANGE THIS PART FOR DIFFERENT MODELS
unit = 1
width = 10
length = int(2 * width / unit)
num_test = 200


total_idx = [x+1 for x in list(range(3000))]
    # set seed so that every model will use the same test set
np.random.seed(7)
test_idx = [x+1 for x in np.random.choice(3000, num_test, replace=False)]
train_idx = list(set(total_idx).difference(set(test_idx)))

pro_train = []
lig_train = []
label_train = []

for idx in train_idx:
    
    create_data_rec(idx,idx,1,pro_train,lig_train,label_train,width,unit)
    idx_false1 = random.choice(list(set(train_idx) - set([idx])))
        
    create_data_rec(idx,idx_false1,0,pro_train,lig_train,label_train,width,unit)
    #idx_false2 = random.choice(list(set(train_idx) - set([idx, idx_false1])))
        
    #create_data_rec(idx,idx_false2,0,pro_train,lig_train,label_train,width,unit)

pro_train = np.array(pro_train)
lig_train = np.array(lig_train)
X = np.concatenate([pro_train, lig_train], axis=4)
y = np.array(label_train)

In [7]:
import torch

In [8]:
np.array(pro_train).shape

(5600, 20, 20, 20, 2)

In [9]:
np.array(lig_train).shape

(5600, 20, 20, 20, 2)

In [10]:
X.shape

(5600, 20, 20, 20, 4)

In [11]:
X=np.swapaxes(X,1,4)
X=np.swapaxes(X,2,4)
X=np.swapaxes(X,3,4)
X.shape

(5600, 4, 20, 20, 20)

In [12]:
y.shape

(5600,)

In [46]:
y.shape[0]/5

1120.0

In [47]:
y[0:1120]

array([1, 0, 1, ..., 0, 1, 0])

In [13]:
x_train=torch.from_numpy(X[1120:]).float()
x_val=torch.from_numpy(X[0:1120]).float()

In [14]:
y_train=torch.from_numpy(y[1120:]).long()
y_val=torch.from_numpy(y[0:1120]).long()

In [15]:
x_train.shape

torch.Size([4480, 4, 20, 20, 20])

In [16]:
x_val.shape

torch.Size([1120, 4, 20, 20, 20])

In [17]:
y_train.shape

torch.Size([4480])

In [18]:
y_val.shape

torch.Size([1120])

In [19]:
test_cnn = torch.nn.Sequential(
            # Defining a 3D convolution layer
            torch.nn.Conv3d(4, 30, kernel_size=(3,3,3), stride=1, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool3d(kernel_size=(2,2,2), stride=1),
            torch.nn.BatchNorm3d(30),
            
            # Defining a 3D convolution layer
            torch.nn.Conv3d(30, 60, kernel_size=(3,3,3), stride=1, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool3d(kernel_size=(2,2,2), stride=1),
            torch.nn.BatchNorm3d(60),
            
            # Defining a 3D convolution layer
            torch.nn.Conv3d(60, 90, kernel_size=(3,3,3), stride=1, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool3d(kernel_size=(2,2,2), stride=1),
            torch.nn.BatchNorm3d(90),
        )

In [20]:
x=test_cnn(x_train[0:1])
x.view(x.size(0), -1).shape

torch.Size([1, 442170])

In [21]:
y[0:10]

array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [8]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.cnn_layers = torch.nn.Sequential(
            # Defining a 3D convolution layer
            torch.nn.Conv3d(4, 30, kernel_size=(3,3,3), stride=1, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool3d(kernel_size=(2,2,2), stride=1),
            torch.nn.BatchNorm3d(30),
            
            # Defining a 3D convolution layer
            torch.nn.Conv3d(30, 60, kernel_size=(3,3,3), stride=1, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool3d(kernel_size=(2,2,2), stride=1),
            torch.nn.BatchNorm3d(60),
            
            # Defining a 3D convolution layer
            torch.nn.Conv3d(60, 90, kernel_size=(3,3,3), stride=1, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool3d(kernel_size=(2,2,2), stride=1),
            torch.nn.BatchNorm3d(90),
        )

        self.linear_layers = torch.nn.Sequential(
            torch.nn.Linear(442170, 256),
            torch.nn.Dropout(0.4),
            torch.nn.ReLU(),
            torch.nn.Linear(256,2),
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)

        ##Flatten
        x = x.view(x.size(0), -1)

        x = self.linear_layers(x)
        return x

In [23]:
def PrepareTrainData(x,y):
    traindata=[]
    for i in range(0,len(x)):
        traindata.append([x[i],y[i]])
    return traindata

In [24]:
train_data=PrepareTrainData(x_train,y_train)
val_data=PrepareTrainData(x_val,y_val)

In [26]:
import torch.utils.data as Data

In [27]:
train_loader = Data.DataLoader(
    dataset=train_data,      # torch TensorDataset format
    batch_size=20,      
    shuffle=True,              
    num_workers=2,              
)

val_loader = Data.DataLoader(
    dataset=val_data,      # torch TensorDataset format
    batch_size=20,      
    shuffle=True,              
    num_workers=2, 
)


In [16]:
import pandas as pd

In [29]:
model=Model()
print(model)

Model(
  (cnn_layers): Sequential(
    (0): Conv3d(4, 30, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm3d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv3d(30, 60, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool3d(kernel_size=(2, 2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
    (7): BatchNorm3d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Conv3d(60, 90, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool3d(kernel_size=(2, 2, 2), stride=1, padding=0, dilation=1, ceil_mode=False)
    (11): BatchNorm3d(90, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=442170, out_features=2

In [30]:
def Train(epoch,learning_rate,version):

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_func = torch.nn.CrossEntropyLoss()
    epochs=epoch

    epoch_list=[]
    train_loss_list=[]
    val_loss_list=[]
    train_acc_list=[]
    val_acc_list=[]

    best_loss=0

    for e in range(epochs):

        train_loss = 0.0

        train_total = 0
        val_total = 0
        
        train_correct = 0
        val_correct = 0

        for data, labels in train_loader:
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()
            
            optimizer.zero_grad()
            target = model(data)
            loss = loss_func(target,labels.long())
            loss.backward()
            optimizer.step()
            train_loss = loss.item() * data.size(0)

            scores, predictions = torch.max(target.data, 1)
            train_total += labels.size(0)
            train_correct += int(sum(predictions == labels))
            train_acc = round(train_correct / train_total, 2)
        
        valid_loss = 0.0
        ##model.eval()     # Optional when not using Model Specific layer
        for data, labels in val_loader:
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()
            
            target = model(data)
            loss = loss_func(target,labels.long())
            valid_loss = loss.item() * data.size(0) 

            scores, predictions = torch.max(target.data, 1)
            val_total += labels.size(0)
            val_correct += int(sum(predictions == labels))
            val_acc = round(val_correct / val_total, 2)
        
        train_loss=train_loss / len(train_loader)
        valid_loss=valid_loss / len(val_loader)
        print('Epoch {} \t\t Training Loss: {} \t\t Validation Loss: {} \t\t Training Accuracy: {} \t\t Validation Accuracy: {}'.format(e+1,train_loss,valid_loss,train_acc,val_acc))
        epoch_list.append(e+1)
        train_loss_list.append(train_loss)
        val_loss_list.append(valid_loss)
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)


        PATH='mode_{}_epoch_{}.pkl'.format(version,e+1)
        torch.save(model, PATH)

    
    return model,pd.DataFrame({'Epoch':epoch_list,'Training Loss':train_loss_list,'Validation Loss':val_loss_list,'Training Accuracy':train_acc_list,'Validation Accuracy':val_acc_list})

In [31]:
model,loss_df=Train(3,0.01,1)

Epoch 1 		 Training Loss: 2.4265210969107494 		 Validation Loss: 0.8636939525604248 		 Training Accuracy: 0.91 		 Validation Accuracy: 0.94
Epoch 2 		 Training Loss: 0.19200652837753296 		 Validation Loss: 0.6744888850620815 		 Training Accuracy: 0.94 		 Validation Accuracy: 0.95
Epoch 3 		 Training Loss: 0.00034607840851614516 		 Validation Loss: 0.02472087208713804 		 Training Accuracy: 0.94 		 Validation Accuracy: 0.94


In [32]:
loss_df

Epoch  Training Loss  Validation Loss  Training Accuracy  \
0      1       2.426521         0.863694               0.91   
1      2       0.192007         0.674489               0.94   
2      3       0.000346         0.024721               0.94   

   Validation Accuracy  
0                 0.94  
1                 0.95  
2                 0.94

In [33]:
loss_df.to_csv('loss_1.csv')

In [37]:
model_1=torch.load('model_1_epoch_1.pkl')

In [9]:
def get_pred_df_for_pro(pro, lig_list, model, width, unit):
    pro_test, lig_test, label_test = [], [], []
    for lig in lig_list:
        create_data_rec2(pro, lig, -1, pro_test, lig_test, label_test, width, unit)
    testset = np.concatenate([pro_test, lig_test], axis=4)
    
    testset=np.swapaxes(testset,1,4)
    testset=np.swapaxes(testset,2,4)
    testset=np.swapaxes(testset,3,4)
    
    pred = model_2(torch.tensor(testset).float())[:,1]
    d = {'idx': lig_list, 'prob': list(pred.data.numpy())}
    pred_top20 = pd.DataFrame(d).sort_values(by='prob', ascending=False).head(20).set_index('idx')


    return pred_top20

In [18]:
num_test =10
# ============================================
top10_list = []
lig_list = list(range(1, num_test))

for pro in range(1, num_test):
    pred1_top20 = get_pred_df_for_pro(pro, lig_list, model_2, width, unit)

    pred_top20 = pred1_top20
    top10_idx = pred_top20.sort_values(by=['prob'],
                                       ascending=[False]).head(10).index
    print(pro, list(top10_idx))
    top10_list.append([pro] + list(top10_idx))

1 [1, 9, 6, 2, 8, 7, 4, 3, 5]
2 [1, 8, 2, 9, 6, 4, 7, 5, 3]
3 [3, 1, 7, 6, 9, 5, 8, 4, 2]
4 [1, 9, 4, 6, 2, 8, 7, 5, 3]
5 [9, 8, 1, 3, 6, 2, 7, 5, 4]
6 [9, 1, 5, 6, 8, 3, 2, 4, 7]
7 [6, 3, 9, 1, 2, 4, 8, 7, 5]
8 [3, 2, 9, 5, 1, 6, 7, 8, 4]
9 [3, 1, 6, 9, 2, 5, 4, 8, 7]


In [88]:
pred1_top20

prob
idx             
3     388.508209
1     -27.266970
6    -105.058197
9    -139.606613
2    -633.232422
8    -870.421570
5   -1100.527832
4   -1511.351318
7   -1637.160889

In [10]:
DATA_DIR2='E:/Dataset/course/BS6207/testing_data_release/testing_data/'

In [13]:
def create_data_rec2(pro, lig, label,
                    pro_list, lig_list, label_list, width, unit):
    file_name = "{}_{}_cg.pdb".format(("0000" + str(lig))[-4:], "lig")
    X_list, Y_list, Z_list, atomtype_list = read_pdb_test(DATA_DIR2+ file_name)
    x0 = floor(np.mean(X_list))
    y0 = floor(np.mean(Y_list))
    z0 = floor(np.mean(Z_list))
    all_atoms = list(zip(X_list, Y_list, Z_list, atomtype_list))
    use_atoms = [x for x in all_atoms if point_within_vox((x[0], x[1], x[2]),
                                                          x0 - width, x0 + width, y0 - width,
                                                          y0 + width, z0 - width, z0 + width)]
    
    lig_list.append(convert_xyz_to_vox(use_atoms, x0 - width, x0 + width, y0 - width,
                                       y0 + width, z0 - width, z0 + width, unit))
    
    
    
    file_name = "{}_{}_cg.pdb".format(("0000" + str(pro))[-4:], "pro")
    X_list, Y_list, Z_list, atomtype_list = read_pdb_test(DATA_DIR2 + file_name)
    all_atoms = list(zip(X_list, Y_list, Z_list, atomtype_list))
    use_atoms = [x for x in all_atoms if point_within_vox((x[0], x[1], x[2]),
                                                          x0 - width, x0 + width, y0 - width,
                                                          y0 + width, z0 - width, z0 + width)]
    pro_list.append(convert_xyz_to_vox(use_atoms, x0 - width, x0 + width, y0 - width,
                                       y0 + width, z0 - width, z0 + width, unit))
    label_list.append(label)

In [12]:
def read_pdb_test(filename):
	
	with open(filename, 'r') as file:
		strline_L = file.readlines()
		# print(strline_L)

	X_list = list()
	Y_list = list()
	Z_list = list()
	atomtype_list = list()
	for strline in strline_L:

		stripped_line = strline.split()

		line_length = len(stripped_line)
		
		
		X_list.append(float(stripped_line[0]))
		Y_list.append(float(stripped_line[1]))
		Z_list.append(float(stripped_line[2]))

		atomtype = stripped_line[3]
		if atomtype == 'C':
			atomtype_list.append('h') # 'h' means hydrophobic
		else:
			atomtype_list.append('p') # 'p' means polar

	return X_list, Y_list, Z_list, atomtype_list

In [11]:
model_2=torch.load('model_1_epoch_1.pkl')